In [9]:
## Contains IOC for a 2DOF manipulator
## Author : Avadesh Meduri
## Date : 22/02/2022

import numpy as np
import pinocchio as pin
from IPython.display import HTML


from inverse_qp import IOC

from robots import TwoDofManipulator, two_dof_vis

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [2]:
model, data = TwoDofManipulator()
nq, nv = model.nq, model.nv
f_id = model.getFrameId("FOOT")

In [3]:

class DiffFK(Function):
    
    @staticmethod
    def forward(ctx, state, model, data, f_id):
        
        state = state.double().detach().numpy()
        nq, nv = model.nq, model.nv
        pin.forwardKinematics(model, data, state[0:nq], state[nq:nq + nv], np.zeros(nv))
        pin.updateFramePlacements(model, data)
        
        J = pin.computeFrameJacobian(model, data, state[0:nq], f_id, pin.ReferenceFrame.LOCAL)
        J_rot = np.matmul(data.oMf[f_id].rotation,J[0:3])
        
        J_rot_torch = torch.tensor(J_rot, dtype = torch.double)
        ctx.J = J_rot_torch
        
        return torch.tensor(data.oMf[f_id].translation)

    @staticmethod
    def backward(ctx, grad):
        
        jac_rot_torch = ctx.J
        fk_q = jac_rot_torch.t()@grad # derivative wrt joint positions
        fk_dq = torch.zeros(jac_rot_torch.shape[1]) # derivative wrt joint velocities
        
        return torch.hstack((fk_q, fk_dq)), None, None, None
        
    

In [4]:
dfk = DiffFK.apply
def quadratic_loss(q_pred, x_des):
    return torch.linalg.norm(dfk(q_pred[-4:], model, data, f_id) - x_des)

In [11]:
nq = 2
n_col = 10
u_max = [400, 400]
ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0)

lr = 5e-2
eps = 100
optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)

x_init = [np.pi/3.0, -np.pi/4.0, 0, 0]


q_des = np.array([-np.pi/2.0, np.pi/3.0, 0, 0])

pin.forwardKinematics(model, data, q_des[0:nq],q_des[nq:nq+nv], np.zeros(nv))
pin.updateFramePlacements(model, data)

x_des = torch.tensor(data.oMf[f_id].translation)


for i in range(eps):
    
    x_pred = ioc(x_init) 
    
    loss = quadratic_loss(x_pred, x_des)
    print("loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [12]:
x_pred = ioc(x_init) 
x_pred = x_pred.detach().numpy()
q_traj = np.zeros((nq+nv, n_col+1))
for i in range(n_col+1):
    q = x_pred[6*i:6*i+2]
    v = x_pred[6*i+2:6*i+4]
    q_traj[:,i] = np.hstack((q,v))
    pin.forwardKinematics(model, data, q, v, np.zeros(nv))
    pin.updateFramePlacements(model, data)
    

In [13]:
vid = two_dof_vis(q_traj, x_des, model, data)
HTML(vid.to_html5_video())